# Store CSV to Store Segments

This Jupyter Notebook demonstrates how, by using ArcGIS, you can start with little more than a list of stores with sales volume and coordinates, and end with stores segmented by similar demographic characteristics.

1. Prepare Input Data
2. Acquire Demographic Analysis Factors Based on Drive Time Trade Areas Around Stores
3. Segment Stores Using KMeans

### Note: Increased IOPub
For visualization, if you did not start this notebook with an increased data rate limit, stop the notebook, go back to the command line, and start Jupyter Notebook using the following command.

`jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000`

## Prepare Data

The data coming in from the CSV file, while the coordinate locations are present in the data, ArcGIS does not yet know how to recognize the data as _spatial_ for subseqnet analysis steps. To accomplish this, we will load the data into a Pandas DataFrame, convert this into an ArcGIS SpatialDataFrame, and finally create an ArcGIS Feature Set, which we will then use for subsequent analysis.

In [98]:
import pandas as pd
import arcgis

Load the data into a Pandas DataFrame from a CSV file.

In [99]:
df = pd.read_csv('./store_locations.csv', index_col='OBJECTID')
df.head()

,LOCNUM,SALESVOL,X,Y
OBJECTID,,,,
1,666990510,35495,-121.8430,36.6210
2,653371815,35495,-121.8112,36.6676
3,423468472,35495,-121.9651,36.9753
4,511743478,35495,-121.7740,36.9154
5,404459478,52059,-122.0362,37.3231


While the coordinates for each store are contained in an X (longitude) and Y (latitude) field, the data is not yet able to be recognized spatially. We need to create a point geometry for each location in a new field so the data will be recognized as spatial. Once this is done, we also can get rid of the explicity X and Y fields, since the location is now stored in the SHAPE field.

In [100]:
df['SHAPE'] = df.apply(lambda row: arcgis.geometry.Point({'x': row.X, 'y': row.Y, 'spatialReference': {'wkid': 4326}}), axis=1)
df = df.drop(['X', 'Y'], axis=1)
df.head()

,LOCNUM,SALESVOL,SHAPE
OBJECTID,,,
1,666990510,35495,"{'y': 36.621, 'spatialReference': {'wkid': 432..."
2,653371815,35495,"{'y': 36.6676, 'spatialReference': {'wkid': 43..."
3,423468472,35495,"{'y': 36.9753, 'spatialReference': {'wkid': 43..."
4,511743478,35495,"{'y': 36.9154, 'spatialReference': {'wkid': 43..."
5,404459478,52059,"{'y': 37.3231, 'spatialReference': {'wkid': 43..."


Now, with the location data properly formatted to be recoginzed as point geometry, we can create a SpatialDataFrame with the store locations so the data will now be recognized as spatial data.

In [101]:
sdf = arcgis.features.SpatialDataFrame(df)
sdf.set_geometry(col='SHAPE')  # assign the properly formatted shape field to be recognized by the SpatialDataFrame
sdf.reset_index(inplace=True, drop=True)
sdf.head()

,LOCNUM,SALESVOL,SHAPE
0,666990510,35495,"{'y': 36.621, 'spatialReference': {'wkid': 432..."
1,653371815,35495,"{'y': 36.6676, 'spatialReference': {'wkid': 43..."
2,423468472,35495,"{'y': 36.9753, 'spatialReference': {'wkid': 43..."
3,511743478,35495,"{'y': 36.9154, 'spatialReference': {'wkid': 43..."
4,404459478,52059,"{'y': 37.3231, 'spatialReference': {'wkid': 43..."


In [112]:
# get a subset to test with, just the first five records
sdf = sdf[:5]

Convert the SpatailDataFrame to a FeatureSet to use as input for subsequent analysis steps.

__NOTE:__ On 18Aug2017, this will not work unless you have access to the development reposititory, since there was a bug in the `to_featureset` method, which was recently fixed.

In [113]:
fs_store_locations = sdf.to_featureset()
fs_store_locations

{"geometryType": "esriGeometryPoint", "features": [{"geometry": {"spatialReference": {"wkid": 4326}, "y": 36.62100000000007, "x": -121.84299999999992}, "attributes": {"SALESVOL": 35495, "LOCNUM": 666990510}}, {"geometry": {"spatialReference": {"wkid": 4326}, "y": 36.62100000000007, "x": -121.84299999999992}, "attributes": {"SALESVOL": 35495, "LOCNUM": 653371815}}, {"geometry": {"spatialReference": {"wkid": 4326}, "y": 36.62100000000007, "x": -121.84299999999992}, "attributes": {"SALESVOL": 35495, "LOCNUM": 423468472}}, {"geometry": {"spatialReference": {"wkid": 4326}, "y": 36.62100000000007, "x": -121.84299999999992}, "attributes": {"SALESVOL": 35495, "LOCNUM": 511743478}}, {"geometry": {"spatialReference": {"wkid": 4326}, "y": 36.62100000000007, "x": -121.84299999999992}, "attributes": {"SALESVOL": 52059, "LOCNUM": 404459478}}], "fields": []}

## Acquire Demographic Analysis Factors Based on Drive Time Trade Areas Around Stores

Intantiate a Web GIS object instance.

In [104]:
from getpass import getpass

gis_coldbrew = arcgis.gis.GIS(
    url='http://portal.coldbrew.esri.com/portal',
    username='headless', 
    password=getpass('Please enter the headless password: ')
)

Please enter the headless password: ········


## Get Variables for Enrichment from Server

Get the entire list of variables using the get method in the Python API, thus taking care of the access token for us.

In [105]:
resp_get_vars = gis_coldbrew._con.get(
    path='https://ba.coldbrew.esri.com/arcgis/rest/services/DefaultMap/MapServer/exts/BAServer/GetVariables/execute'
)
resp_get_vars

{'messages': [],
 'results': [{'dataType': 'VariableInfo',
   'paramName': 'Variable',
   'value': [{'alias': '2016 Total Population',
     'category': '2016 Key Demographic Indicators (Esri)',
     'countryAbbrev': 'US',
     'datasetID': 'USA_ESRI_2016',
     'decimals': 0,
     'description': '2016 Total Population (Esri)',
     'name': 'TOTPOP_CY',
     'units': 'count'},
    {'alias': '2016 Population in Households',
     'category': '2016 Key Demographic Indicators (Esri)',
     'countryAbbrev': 'US',
     'datasetID': 'USA_ESRI_2016',
     'decimals': 0,
     'description': '2016 Household Population (Esri)',
     'name': 'HHPOP_CY',
     'percentage': 'TOTPOP_CY',
     'percentageAlias': '2016 Total Population',
     'units': 'count'},
    {'alias': '2016 Population in Families',
     'category': '2016 Key Demographic Indicators (Esri)',
     'countryAbbrev': 'US',
     'datasetID': 'USA_ESRI_2016',
     'decimals': 0,
     'description': '2016 Family Population (Esri)',
     '

In [11]:
fs_service_area = arcgis.features.FeatureSet(result_service_area['saPolygons']['features'])

Convert the JSON response to a Pandas DataFrame filtered to just use the fields with names ending in `_CY` for current year variabes, which are the basic descriptive demographics. Using the DataFrame makes getting summarized values a little easier.

In [106]:
df = pd.DataFrame([field for field in resp_get_vars['results'][0]['value'] if field['name'].endswith('_CY')])
df.head()

,alias,category,countryAbbrev,datasetID,decimals,description,indexBase,name,percentage,percentageAlias,units
0,2016 Total Population,2016 Key Demographic Indicators (Esri),US,USA_ESRI_2016,0,2016 Total Population (Esri),NaN,TOTPOP_CY,NaN,NaN,count
1,2016 Population in Households,2016 Key Demographic Indicators (Esri),US,USA_ESRI_2016,0,2016 Household Population (Esri),NaN,HHPOP_CY,TOTPOP_CY,2016 Total Population,count
2,2016 Population in Families,2016 Key Demographic Indicators (Esri),US,USA_ESRI_2016,0,2016 Family Population (Esri),NaN,FAMPOP_CY,TOTPOP_CY,2016 Total Population,count
3,2016 Population in Group Quarters,2016 Key Demographic Indicators (Esri),US,USA_ESRI_2016,0,2016 Group Quarters Population (Esri),NaN,GQPOP_CY,TOTPOP_CY,2016 Total Population,count
4,2016 Population Density,2016 Key Demographic Indicators (Esri),US,USA_ESRI_2016,1,2016 Population Density (Pop per Square Mile) ...,NaN,POPDENS_CY,NaN,NaN,count


Using the `unique` method to get a list of unique variable names, along with the enrichment categories.

In [107]:
enrichment_variables = df.name.unique().tolist()
enrichment_variables

['TOTPOP_CY',
 'HHPOP_CY',
 'FAMPOP_CY',
 'GQPOP_CY',
 'POPDENS_CY',
 'TOTHH_CY',
 'AVGHHSZ_CY',
 'FAMHH_CY',
 'AVGFMSZ_CY',
 'TOTHU_CY',
 'OWNER_CY',
 'RENTER_CY',
 'VACANT_CY',
 'POP0_CY',
 'POP5_CY',
 'POP10_CY',
 'POP15_CY',
 'POP20_CY',
 'POP25_CY',
 'POP30_CY',
 'POP35_CY',
 'POP40_CY',
 'POP45_CY',
 'POP50_CY',
 'POP55_CY',
 'POP60_CY',
 'POP65_CY',
 'POP70_CY',
 'POP75_CY',
 'POP80_CY',
 'POP85_CY',
 'POP18UP_CY',
 'POP21UP_CY',
 'MEDAGE_CY',
 'MALES_CY',
 'MALE0_CY',
 'MALE5_CY',
 'MALE10_CY',
 'MALE15_CY',
 'MALE20_CY',
 'MALE25_CY',
 'MALE30_CY',
 'MALE35_CY',
 'MALE40_CY',
 'MALE45_CY',
 'MALE50_CY',
 'MALE55_CY',
 'MALE60_CY',
 'MALE65_CY',
 'MALE70_CY',
 'MALE75_CY',
 'MALE80_CY',
 'MALE85_CY',
 'MAL18UP_CY',
 'MAL21UP_CY',
 'MEDMAGE_CY',
 'FEMALES_CY',
 'FEM0_CY',
 'FEM5_CY',
 'FEM10_CY',
 'FEM15_CY',
 'FEM20_CY',
 'FEM25_CY',
 'FEM30_CY',
 'FEM35_CY',
 'FEM40_CY',
 'FEM45_CY',
 'FEM50_CY',
 'FEM55_CY',
 'FEM60_CY',
 'FEM65_CY',
 'FEM70_CY',
 'FEM75_CY',
 'FEM80_CY',
 'F

In [108]:
enrichment_categories = df.category.unique().tolist()
enrichment_categories

['2016 Key Demographic Indicators (Esri)',
 '2016 Age: 5 Year Increments (Esri)',
 '2016 Age: 1 Year Increments (Esri)',
 '2016 Labor Force by Industry (Esri)',
 '2016 Labor Force by Occupation (Esri)',
 '2016 Race and Hispanic Origin (Esri)',
 '2016 Educational Attainment (Esri)',
 '2016 Marital Status (Esri)',
 '2016 Income (Esri)',
 '2016 Income by Age (Esri)',
 '2016 Disposable Income (Esri)',
 '2016 Disposable Income by Age (Esri)',
 '2016 Net Worth (Esri)',
 '2016 Net Worth by Age (Esri)',
 '2016 Home Value (Esri)',
 '2016/2021 Other (Not Mappable) (Esri)']

## Perform Geoenrichment

Since the ArcGIS Python API requires a published layer to use the built in Geoenrichment method, we utilize the ArcGIS Python API's built in `post` method, which takes care of the token authetication, and also has the `urllib.encode` method built in for converting the payload from a dictionary for the post call.

In [132]:
trade_area_drive_time = 8  # in minutes
study_area_options = '{"areaType":"DriveTimeBuffer","bufferUnits":"esriDriveTimeUnitsMinutes",' + \
        '"bufferRadii":' + '[{drive_time}]'.format(drive_time=trade_area_drive_time) + '}"'
study_area_options = '{"areaType":"DriveTimeBuffer","bufferUnits":"esriDriveTimeUnitsMinutes","bufferRadii":[5]}'

In [133]:
url_geoenrich = gis_coldbrew.properties.helperServices.geoenrichment.url + "/Geoenrichment/Enrich"
payload = {
    'studyAreas': fs_store_locations.features,
#    'analysisVariables': enrichment_variables,
    'dataCollections': '["KeyUSFacts"]',
    'studyAreasOptions': study_area_options,
    'f': 'json'
}
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
}
resp_enrich = gis_coldbrew._con.post(url_geoenrich, postdata=payload)
resp_enrich

{'messages': [{'description': 'Travel modes are ignored since they are not available for Service Area Solver.',
   'id': '-1',
   'type': 'esriJobMessageTypeWarning'},
  {'description': '{"code":400,"messageCode":"CONT_0001","message":"Item does not exist or is inaccessible.","details":[]}',
   'id': '-1',
   'type': 'esriJobMessageTypeError'}],
 'results': [{'dataType': 'GeoEnrichmentResult',
   'paramName': 'GeoEnrichmentResult',
   'value': {'FeatureSet': [], 'version': '0.3'}}]}

In [89]:
response_feature_set = response['results'][0]['value']['FeatureSet'][0]
response_feature_set

{'displayFieldName': '',
 'features': [{'attributes': {'AREA_ID': '0_1',
    'AVGHHSZ_CY': 3.44,
    'AVGHINC_CY': 70964,
    'AVGHINC_FY': 78046,
    'AVGVAL_CY': 491686,
    'AVGVAL_FY': 544829,
    'DIVINDX_CY': 90.1,
    'FAMGRW10CY': 0.37,
    'FAMGRWCYFY': 0.53,
    'GQPOP_CY': 68,
    'HHGRW10CY': 0.39,
    'HHGRWCYFY': 0.54,
    'HasData': 1,
    'ID': '0',
    'LOCNUM': 666990510,
    'MEDHINC_CY': 54029,
    'MEDHINC_FY': 58430,
    'MEDVAL_CY': 462703,
    'MEDVAL_FY': 527755,
    'MHIGRWCYFY': 1.58,
    'OBJECTID': 1,
    'OWNER_CY': 1768,
    'OWNER_FY': 1813,
    'PCIGRWCYFY': 1.81,
    'PCI_CY': 21218,
    'PCI_FY': 23204,
    'POPGRW10CY': 0.59,
    'POPGRWCYFY': 0.65,
    'RENTER_CY': 2402,
    'RENTER_FY': 2470,
    'SALESVOL': 35495,
    'TOTHH00': 3918,
    'TOTHH10': 4070,
    'TOTHH_CY': 4170,
    'TOTHH_FY': 4283,
    'TOTHU00': 4301,
    'TOTHU10': 4275,
    'TOTHU_CY': 4426,
    'TOTHU_FY': 4553,
    'TOTPOP00': 13667,
    'TOTPOP10': 13903,
    'TOTPOP_CY': 14

In [115]:
fs_enrich = arcgis.features.FeatureSet(
    features=resp_enrich['results'][0]['value']['FeatureSet'][0]['features'], 
    fields=resp_enrich['results'][0]['value']['FeatureSet'][0]['fields']
)
fs_enrich

{"features": [{"attributes": {"ID": "0", "sourceCountry": "US", "RENTER_FY": 2470, "OBJECTID": 1, "SALESVOL": 35495, "TOTHU_CY": 4426, "RENTER_CY": 2402, "TOTHH_FY": 4283, "GQPOP_CY": 68, "TOTPOP_CY": 14420, "POPGRW10CY": 0.59, "MEDHINC_FY": 58430, "TOTHU00": 4301, "DIVINDX_CY": 90.1, "TOTHH00": 3918, "HHGRWCYFY": 0.54, "AVGVAL_FY": 544829, "TOTHU_FY": 4553, "TOTHU10": 4275, "AVGHHSZ_CY": 3.44, "VACANT_FY": 270, "bufferUnits": "esriMiles", "PCIGRWCYFY": 1.81, "HasData": 1, "aggregationMethod": "BlockApportionment:US.BlockGroups", "TOTHH_CY": 4170, "MHIGRWCYFY": 1.58, "AVGHINC_FY": 78046, "POPGRWCYFY": 0.65, "TOTPOP00": 13667, "VACANT_CY": 256, "TOTPOP_FY": 14896, "MEDHINC_CY": 54029, "bufferUnitsAlias": "mile", "AVGVAL_CY": 491686, "PCI_CY": 21218, "FAMGRW10CY": 0.37, "MEDVAL_CY": 462703, "OWNER_CY": 1768, "TOTHH10": 4070, "FAMGRWCYFY": 0.53, "OWNER_FY": 1813, "bufferRadii": 1, "TOTPOP10": 13903, "AREA_ID": "0_1", "areaType": "RingBuffer", "PCI_FY": 23204, "MEDVAL_FY": 527755, "HHGRW10

In [46]:
[field['name'] for field in response_feature_set.fields]

['OBJECTID',
 'areaType',
 'bufferRadii',
 'bufferUnits',
 'bufferUnitsAlias',
 'ID',
 'LOCNUM',
 'SALESVOL',
 'sourceCountry',
 'AREA_ID',
 'HasData',
 'aggregationMethod',
 'AVGHHSZ_CY',
 'AVGHINC_CY',
 'AVGHINC_FY',
 'AVGVAL_CY',
 'AVGVAL_FY',
 'DIVINDX_CY',
 'FAMGRW10CY',
 'FAMGRWCYFY',
 'GQPOP_CY',
 'HHGRW10CY',
 'HHGRWCYFY',
 'MEDHINC_CY',
 'MEDHINC_FY',
 'MEDVAL_CY',
 'MEDVAL_FY',
 'MHIGRWCYFY',
 'OWNER_CY',
 'OWNER_FY',
 'PCI_CY',
 'PCI_FY',
 'PCIGRWCYFY',
 'POPGRW10CY',
 'POPGRWCYFY',
 'RENTER_CY',
 'RENTER_FY',
 'TOTHH00',
 'TOTHH10',
 'TOTHH_CY',
 'TOTHH_FY',
 'TOTHU00',
 'TOTHU10',
 'TOTHU_CY',
 'TOTHU_FY',
 'TOTPOP00',
 'TOTPOP10',
 'TOTPOP_CY',
 'TOTPOP_FY',
 'VACANT_CY',
 'VACANT_FY']

In [116]:
df_enrich = fs_enrich.df
df_enrich

,AREA_ID,AVGHHSZ_CY,AVGHINC_CY,AVGHINC_FY,AVGVAL_CY,AVGVAL_FY,DIVINDX_CY,FAMGRW10CY,FAMGRWCYFY,GQPOP_CY,...,TOTPOP_CY,TOTPOP_FY,VACANT_CY,VACANT_FY,aggregationMethod,areaType,bufferRadii,bufferUnits,bufferUnitsAlias,sourceCountry
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,0_1,3.44,70964,78046,491686,544829,90.1,0.37,0.53,68,...,14420,14896,256,270,BlockApportionment:US.BlockGroups,RingBuffer,1,esriMiles,mile,US
2,1_1,3.44,70964,78046,491686,544829,90.1,0.37,0.53,68,...,14420,14896,256,270,BlockApportionment:US.BlockGroups,RingBuffer,1,esriMiles,mile,US
3,2_1,3.44,70964,78046,491686,544829,90.1,0.37,0.53,68,...,14420,14896,256,270,BlockApportionment:US.BlockGroups,RingBuffer,1,esriMiles,mile,US
4,3_1,3.44,70964,78046,491686,544829,90.1,0.37,0.53,68,...,14420,14896,256,270,BlockApportionment:US.BlockGroups,RingBuffer,1,esriMiles,mile,US
5,4_1,3.44,70964,78046,491686,544829,90.1,0.37,0.53,68,...,14420,14896,256,270,BlockApportionment:US.BlockGroups,RingBuffer,1,esriMiles,mile,US
